In [48]:
import sys
import ezdxf

try:
    doc = ezdxf.readfile("arm5_left_5mm.dxf")
except IOError:
    print(f"Not a DXF file or a generic I/O error.")
    sys.exit(1)
except ezdxf.DXFStructureError:
    print(f"Invalid or corrupted DXF file.")
    sys.exit(2)
    
msp = doc.modelspace()


# helper function
def print_entity(e):
    print("LINE on layer: %s\n" % e.dxf.layer)
    print("start point: %s\n" % e.dxf.start)
    print("end point: %s\n" % e.dxf.end)

# iterate over all entities in modelspace
msp = doc.modelspace()
# print(msp)
for e in msp:
    print(e)
#     if e.dxftype() == "CIRCLE":
#         print_entity(e)

# entity query for all LINE entities in modelspace
# for e in msp.query("SPLINE"):
#     print_entity(e)


CIRCLE(#100)
CIRCLE(#101)
CIRCLE(#102)
LWPOLYLINE(#103)
CIRCLE(#104)
CIRCLE(#105)
CIRCLE(#106)
CIRCLE(#107)
CIRCLE(#108)
LWPOLYLINE(#109)


In [91]:
import sys
import ezdxf

def calculate_dimensions(entity):
    if entity.dxftype() == 'LINE':
        start_point = entity.dxf.start
        end_point = entity.dxf.end
        width = abs(end_point[0] - start_point[0])
        height = abs(end_point[1] - start_point[1])
        position = ((start_point[0] + end_point[0]) / 2, (start_point[1] + end_point[1]) / 2)
    elif entity.dxftype() == 'LWPOLYLINE':
        vertices = entity.get_points()
        min_x = min(vertex[0] for vertex in vertices)
        min_y = min(vertex[1] for vertex in vertices)
        max_x = max(vertex[0] for vertex in vertices)
        max_y = max(vertex[1] for vertex in vertices)
        width = max_x - min_x
        height = max_y - min_y
#         position = ((min_x + max_x) / 2, (min_y + max_y) / 2)
    elif entity.dxftype() == 'CIRCLE':
        center = entity.dxf.center
        diameter = entity.dxf.radius * 2
        width = height = diameter
        min_x = center[0] - diameter/2
        max_x = center[0] + diameter/2
        min_y = center[1] - diameter/2
        max_y = center[1] + diameter/2
        position = center
    elif entity.dxftype() == 'SPLINE':
        # Assuming spline has control points (fit points)
        vertices = entity.control_points()
        min_x = min(vertex[0] for vertex in vertices)
        min_y = min(vertex[1] for vertex in vertices)
        max_x = max(vertex[0] for vertex in vertices)
        max_y = max(vertex[1] for vertex in vertices)
        width = max_x - min_x
        height = max_y - min_y
    elif entity.dxftype() == 'TEXT':
        # For TEXT entities, consider the bounding box of the text
        width = entity.dxf.width
        height = entity.dxf.height
        insertion_point = entity.dxf.insert
        min_x = insertion_point[0] - width / 2
        max_x = insertion_point[0] + width / 2
        min_y = insertion_point[1] - height / 2
        max_y = insertion_point[1] + height / 2
    # Add handling for other entity types as needed
    else:
        # If the entity type is not recognized, return None for dimensions
        width = height = position = min_x = max_x = min_y = max_y =None
    return width, height, min_x, max_x, min_y, max_y


try:
    doc = ezdxf.readfile("left_motor_tensioning_acrylic_5mm.dxf") # Alu_plate.dxf ; arm5_left_5mm.dxf  ; Arm3_acrylic_10mm
except IOError:
    print(f"Not a DXF file or a generic I/O error.")
    sys.exit(1)
except ezdxf.DXFStructureError:
    print(f"Invalid or corrupted DXF file.")
    sys.exit(2)
    
msp = doc.modelspace()

list_min_x = []
list_max_x = []
list_min_y = []
list_max_y = []

# Iterate over all entities in modelspace
for entity in msp:
    width, height, min_x, max_x, min_y, max_y = calculate_dimensions(entity)
    list_min_x.append(min_x) 
    list_max_x.append(max_x) 
    list_min_y.append(min_y) 
    list_max_y.append(max_y)  
#     if width is not None and height is not None:
#         print(f"Entity type: {entity.dxftype()}, Dimensions: {width} x {height}, {min_x}, {max_x}, {min_y}, {max_y}")
bbox = [min(list_min_x),min(list_min_y), max(list_max_x), max(list_max_y)]
# print(bbox) # Corners of th box
area = (bbox[2] - bbox[0]) * (bbox[3]-bbox[1])
print(area) # in mm2

7189.000000000005
